In [75]:
import jax
import pymc as pm

In [76]:
N = 10_000

In [77]:
with pm.Model() as model:
    pm.Normal("x", shape=N)

In [78]:
%%time
with model:
    tr_numpyro = pm.sample(
        nuts_sampler="numpyro",
        discard_tuned_samples=False,
        chains=1,
        idata_kwargs=dict(save_warmup=True),
    )

sample: 100%|██████████| 2000/2000 [00:24<00:00, 83.17it/s, 31 steps of size 1.55e-01. acc. prob=0.85]
Only one chain was sampled, this makes it impossible to run some convergence checks


CPU times: user 24.9 s, sys: 792 ms, total: 25.7 s
Wall time: 24.5 s


In [79]:
tr_numpyro.sample_stats.num_steps.sum("draw")

AttributeError: 'Dataset' object has no attribute 'num_steps'

In [ ]:
%env ORT_DYLIB_PATH=/home/adr/.cache/ort.pyke.io/dfbin/x86_64-unknown-linux-gnu/C23A69F709DF91D2BB185D76A29901BD4BE81CD66D85DBCFC5E8BCD851DE9891/onnxruntime/lib/libonnxruntime.so

import nutpie
import torch
import io

In [ ]:
N = 10_000

In [ ]:
class Module(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.mu = torch.nn.Parameter(torch.ones(N), requires_grad=False)

    def forward(self, x):
        return ((-((x - self.mu) ** 2).sum() / 2), (self.mu - x))
        
mod = Module()

In [ ]:
exported = torch.onnx.dynamo_export(mod, torch.zeros(N))

exported_bytes = io.BytesIO()
exported.save(exported_bytes)
exported_bytes = exported_bytes.getvalue()

providers = nutpie._lib.OnnxProviders()
providers.add_cuda()
#providers.add_cpu()

nutpie_model = nutpie._lib.OnnxModel(N, exported_bytes, providers)

In [ ]:
%%time
settings = nutpie._lib.PyDiagGradNutsSettings()
settings.num_chains = 4
sampler = nutpie._lib.PySampler.from_onnx(settings, 4, nutpie_model, "", 0, callback=None)
tr = sampler.wait()

In [ ]:
tr = sampler.extract_results()

In [ ]:
tr[0][1].field("n_steps").sum()